In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from cartpole_environment import CartPoleEnv

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995
ENV_NAME = 'CartPole-v1'

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    #env = gym.make(ENV_NAME)
    env = CartPoleEnv()
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            #env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                print("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))
                break
            dqn_solver.experience_replay()


if __name__ == "__main__":
    cartpole()

C:\Users\nikma\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Run: 1, exploration: 1.0, score: 16
Run: 2, exploration: 0.946354579813443, score: 15
Run: 3, exploration: 0.7901049725470279, score: 37
Run: 4, exploration: 0.7477194593032545, score: 12
Run: 5, exploration: 0.7111635524897149, score: 11
Run: 6, exploration: 0.6662995813682115, score: 14
Run: 7, exploration: 0.5967292370047992, score: 23
Run: 8, exploration: 0.5590843898207511, score: 14
Run: 9, exploration: 0.5264466124450268, score: 13
Run: 10, exploration: 0.4858739637363176, score: 17
Run: 11, exploration: 0.4529463432347434, score: 15
Run: 12, exploration: 0.4286478463299511, score: 12
Run: 13, exploration: 0.40974000909221303, score: 10
Run: 14, exploration: 0.3877593341372176, score: 12
Run: 15, exploration: 0.36512303261753626, score: 13
Run: 16, exploration: 0.3472722151889232, score: 11
Run: 17, exploration: 0.32864265128599696, score: 12
Run: 18, exploration: 0.3125753549412418, score: 11
Run: 19, exploration: 0.2987875242397482, score: 10
Run: 20, exploration: 0.2856078805

KeyboardInterrupt: 